# Keras

* `tf.contrib.keras` 를 쓰니 좀 이런저런 문제가 있어서 따로 인스톨해서 사용함

* keras 를 이용해서 tutorial 을 구현해보자.
* keras 를 이용해서 transfer learning 을 구현해보자.
* 추가 - slim 을 이용해서 transfer learning 을 구현해보자.

## TF Tutorial using Keras

* BN model

In [1]:
# from keras.datasets import mnist
from tensorflow.examples.tutorials.mnist import input_data
import keras
from keras import backend as K
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Dense, Flatten, Dropout
import numpy as np

Using TensorFlow backend.


In [2]:
keras.__version__

'2.0.6'

In [3]:
# GPU memory control in keras
# import tensorflow as tf
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
# K.tensorflow_backend.set_session(tf.Session(config=config))

In [4]:
np.random.seed(0)

In [5]:
# (x_tv, y_tv), (x_test, y_test) = mnist.load_data()
# x_train, x_valid = x_tv[:55000], x_tv[55000:]
# y_train, y_valid = y_tv[:55000], y_tv[55000:]

In [6]:
mnist = input_data.read_data_sets('./MNIST_data', one_hot=True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [7]:
# Keras 에서도 mnist 를 제공하지만 데이터셋을 동일하게 맞추기 위해 tf 의 mnist 를 사용하자.
X_train, y_train = mnist.train.images, mnist.train.labels
X_valid, y_valid = mnist.validation.images, mnist.validation.labels
X_test, y_test = mnist.test.images, mnist.test.labels

In [8]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_valid = X_valid.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

## Sequential API

In [9]:
# Sequential model
from keras.models import Sequential

In [10]:
def build_model_sequential(input_shape, output_units):
    model = Sequential()

    n_filters = 32
    for i in range(3):
        if i == 0:
            model.add(Conv2D(n_filters, [3,3], use_bias=False, padding='same', input_shape=input_shape))
        else:
            model.add(Conv2D(n_filters, [3,3], use_bias=False, padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Conv2D(n_filters, [3,3], use_bias=False, padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling2D())
        model.add(Dropout(0.3))
        
        n_filters *= 2

    model.add(Flatten())
    model.add(Dense(output_units, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [11]:
model = build_model_sequential([28, 28, 1], 10)

In [12]:
result = model.fit(X_train, y_train, batch_size=128, epochs=100, verbose=1, validation_data=(X_test, y_test))

Train on 55000 samples, validate on 10000 samples
Epoch 1/100
55000/55000 [==============================] - 9s - loss: 0.2186 - acc: 0.9312 - val_loss: 0.4587 - val_acc: 0.8397
Epoch 2/100
55000/55000 [==============================] - 8s - loss: 0.0593 - acc: 0.9813 - val_loss: 0.0269 - val_acc: 0.9909
Epoch 3/100
55000/55000 [==============================] - 8s - loss: 0.0477 - acc: 0.9848 - val_loss: 0.0269 - val_acc: 0.9917
Epoch 4/100
55000/55000 [==============================] - 8s - loss: 0.0366 - acc: 0.9884 - val_loss: 0.0222 - val_acc: 0.9928
Epoch 5/100
55000/55000 [==============================] - 8s - loss: 0.0351 - acc: 0.9889 - val_loss: 0.0233 - val_acc: 0.9917
Epoch 6/100
55000/55000 [==============================] - 8s - loss: 0.0302 - acc: 0.9898 - val_loss: 0.0193 - val_acc: 0.9937
Epoch 7/100
55000/55000 [==============================] - 8s - loss: 0.0265 - acc: 0.9916 - val_loss: 0.0250 - val_acc: 0.9920
Epoch 8/100
47360/55000 [========================>....

KeyboardInterrupt: 

In [ ]:
print "{:.2%}".format(np.average(result.history['val_acc'][-5:]))

## Functional API

시퀀셜 모델과 동일하나 펑셔널 API 가 더 flexible 한 듯 함.

In [ ]:
from keras.layers import Input
from keras.models import Model

In [ ]:
def build_model_functional(input_shape, output_units):
    input_tensor = Input(input_shape)
    net = input_tensor
    n_filters = 32

    for _ in range(3):
        net = Conv2D(n_filters, [3,3], padding='same')(net)
        net = BatchNormalization()(net)
        net = Activation('relu')(net)
        net = Conv2D(n_filters, [3,3], padding='same')(net)
        net = BatchNormalization()(net)
        net = Activation('relu')(net)
        net = MaxPooling2D()(net)
        net = Dropout(0.3)(net)
        
        n_filters *= 2

    net = Flatten()(net)
    net = Dense(output_units, activation='softmax')(net)

    model = Model(input_tensor, net)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
K.clear_session()

model = build_model_functional([28, 28, 1], 10)

In [ ]:
result = model.fit(X_train, y_train, batch_size=128, epochs=100, verbose=1, validation_data=(X_test, y_test))

In [ ]:
print "{:.2%}".format(np.average(result.history['val_acc'][-5:]))

## Data augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
K.clear_session()

model = build_model_functional(input_tensor=Input([28, 28, 1]))

In [ ]:
datagen = ImageDataGenerator(rotation_range=15, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1)
datagen.fit(X_train)

In [ ]:
result = model.fit_generator(datagen.flow(X_train, y_train, batch_size=128), steps_per_epoch=X_train.shape[0] // 128,
                             epochs=100, validation_data=(X_test, y_test))

In [ ]:
print "{:.2%}".format(np.average(result.history['val_acc'][-5:]))